In [365]:
import pandas as pd
import numpy as np
import copy
import os
%config Completer.use_jedi = False

In [465]:
# generate list of files
PATH = 'D:\\MyDocuments\\Statements\\HSBC\\'
result = [os.path.join(dp, f)
          for dp, dn, filenames in os.walk(PATH) 
          for f in filenames 
          if (os.path.splitext(f)[1] == '.csv') and ('_processed' not in os.path.splitext(f)[0])]

In [466]:
def process_df(df_param):
    df = copy.deepcopy(df_param)
    header_cols = ['date', 'transaction_type', 'payee', 'outflow', 'inflow', 'balance']
    
    # set columns
    df.columns = header_cols
    
    # transform date
    df[['payee', 'outflow', 'inflow', 'balance']] = df[['payee', 'outflow', 'inflow', 'balance']].astype(str)
    df['date'] = pd.to_datetime(
        df['date'].str[:2] 
        + '-' 
        + df['date'].str[3:6] 
        + '-' 
        + '20' + df['date'].str[-2:]
    )
    
    # remove unneeded columns
    cols = ['date', 'payee', 'outflow', 'inflow', 'balance']
    df = df[cols]
    df.reset_index(drop=True, inplace=True)
    
    # change data types
    df.loc[:, 'balance'] = df['balance'].str.replace(',', '')
    df.loc[:, 'outflow'] = df['outflow'].str.replace(',', '')
    df.loc[:, 'inflow'] = df['inflow'].str.replace(',', '')
    df.loc[df['outflow'] == '.', 'outflow'] = 0.0
    df = df.astype({'outflow': float, 'inflow': float, 'balance': float})
    
    df['outflow'].fillna(0, inplace=True)
    df['inflow'].fillna(0, inplace=True)
    df['balance'].fillna(0, inplace=True)
    
    # remove balance carried forward rows within dataset
    df_temp = df.iloc[1:-1, :]
    idx = df_temp.loc[df_temp['payee'].str.contains('(?i)balance'), :].index
    df.drop(idx, inplace=True)
    
    # combine outflow and inflow into single amount column
    df.loc[:, 'outflow'] = (df['outflow']*-1) + df['inflow']
    df = df.rename(columns={'outflow': 'amount'})
    cols = ['date', 'payee', 'amount', 'balance']
    df = df[cols]
    
    # combine multi-line payees
    for idx, row in df.iterrows():
        if ((df.loc[idx, 'amount']==0.0) and (df.loc[idx, 'balance']==0.0)):
            df.loc[idx+1, 'payee'] = df.loc[idx, 'payee'] + ' ' + df.loc[idx+1, 'payee']
            df.loc[idx+1, 'date'] = df.loc[idx, 'date']
    df.drop(df[(df['amount']==0.0) & (df['balance']==0.0)].index, inplace=True)
    df['date'].fillna(method='ffill', inplace=True)
    
    # validate transactions, by comparing balances
    df_datewise = df.groupby('date').sum(['amount', 'balance'])
    df_datewise.reset_index(inplace=True)
    for idx, row in df_datewise.iterrows():
        if idx <= len(df_datewise.index) - 2: 
            df_datewise.loc[idx+1, 'calculated_balance'] = df_datewise.loc[idx, 'balance'] + df_datewise.loc[idx+1, 'amount']
            
    df_datewise.loc[0, 'balanced'] = True
    df_datewise.loc[abs(round(df_datewise['balance'], 2) - round(df_datewise['calculated_balance'], 2)) == 0, 'balanced'] = True
    
    # compare calculated balance with actual balance
    x = len(df_p_d[df_p_d['balanced'] != True].index)
    if x == 0:
        valid = True
    else:
        valid = False
    return df, df_datewise, valid

In [467]:
# validate files
for f in result:
    file_path = f
    print('File: [{file}]'.format(file = file_path))
    df_in = pd.read_csv(file_path, header=None)
    df, df_datewise, valid = process_df(df_in)
    if not valid:
        print('Valid: [{valid}]'.format(valid = valid))

File: [D:\MyDocuments\Statements\HSBC\2019\2019-01-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-02-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-03-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-04-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-05-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-06-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-07-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-08-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-09-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-10-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-11-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2019\2019-12-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2020\2020-01-25_Statement.csv]
File: [D:\MyDocuments\Statements\HSBC\2020\2020-02-25_Statement.csv]
File: [D:\MyDocuments\Statements\H

In [468]:
# process csv files
for f in result:
    file_path = f
    df_in = pd.read_csv(file_path, header=None)
    df, df_datewise, valid = process_df(df_in)
    file_name = os.path.basename(file_path)
    processed_filename = os.path.splitext(file_name)[0] + '_processed.csv'
    processed_full_path = os.path.join(os.path.dirname(file_path), 'processed', processed_filename)
    df.to_csv(processed_full_path, index=False)

In [471]:
# generate list of processed files
result_ynab = [os.path.join(dp, f)
               for dp, dn, filenames in os.walk(PATH) 
              for f in filenames 
              if (os.path.splitext(f)[1] == '.csv') and ('_processed' in os.path.splitext(f)[0])]

In [472]:
def transform_payee(in_str: str) -> (str, str):
    out_str = in_str
    category = ""
    mapping = [{"str": 'asda', 'repl': 'Asda', 'category': 'Basics: Groceries'}
               , {"str": 'tesco', 'repl': 'Tesco', 'category': 'Basics: Groceries'}
               , {"str": 'sainsburys', 'repl': 'Sainsburys', 'category': 'Basics: Groceries'}
               , {"str": 'morrisons', 'repl': 'Morrisons', 'category': 'Basics: Groceries'}
               , {"str": 'dominos', 'repl': 'Dominos', 'category': 'Discretionary: Entertainment'}
               , {"str": 'papa johns', 'repl': 'Papa Johns', 'category': 'Discretionary: Entertainment'}
               , {"str": 'just-eat', 'repl': 'Just Eat', 'category': 'Discretionary: Entertainment'}
               , {"str": 'just-eat', 'repl': 'Just Eat', 'category': 'Discretionary: Entertainment'}
               , {"str": 'netflix', 'repl': 'Netflix', 'category': 'Monthly: Streaming'}
               , {"str": 'amazonprime', 'repl': 'Amazon Prime', 'category': 'Monthly: Streaming'}
               , {"str": 'amazon prime', 'repl': 'Amazon Prime', 'category': 'Monthly: Streaming'}
               , {"str": 'amazon.co.uk', 'repl': 'Amazon', 'category': 'Discretionary: Misc Purchases'}
               , {"str": 'ee limited', 'repl': 'EE Limited', 'category': 'Basics: Phone'}
               , {"str": 'the gym ltd', 'repl': 'The Gym Ltd', 'category': 'Monthly: Gym'}
               , {"str": 's j p fish', 'repl': in_str, 'category': 'Basics: Rent'}
              ]
    for search_string in mapping:
        if search_string['str'] in in_str.lower():
            out_str = search_string['repl']
            category = search_string['category']
        
    if 'pizza' in in_str.lower() and 'hut' in in_str.lower():
        out_str = 'Pizza Hut'
        category = 'Discretionary: Entertainment'
    elif 'papa' in in_str.lower() and 'john' in in_str.lower():
        out_str = 'Papa Johns'
        category = 'Discretionary: Entertainment'
        
    return out_str, category

In [511]:
def process_ynab(df_param):
    df_ynab = copy.deepcopy(df_param)
    
    # rename columns
    df_ynab = df_ynab.rename(columns={'date': 'Date', 'payee': 'Payee'})
    df_ynab['Date'] = pd.to_datetime(df_ynab['Date'])
    
    # remove balance rows
    idx = df_ynab.loc[df_ynab['Payee'].str.contains('(?i)balance'), :].index
    df_ynab.drop(idx, inplace=True)
    
    # create additional ynab columns, format values
    df_ynab['Category'] = ''
    df_ynab['Memo'] = ''
    df_ynab.loc[df_ynab['amount'] < 0, 'Outflow'] = df_ynab['amount'] * -1
    df_ynab.loc[df_ynab['amount'] > 0, 'Inflow'] = df_ynab['amount']
    df_ynab['Outflow'].fillna('', inplace=True)
    df_ynab['Inflow'].fillna('', inplace=True)
    cols = ['Date', 'Payee', 'Category', 'Memo', 'Outflow', 'Inflow']
    df_ynab = df_ynab[cols]
    df_ynab['Date'] = df_ynab['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    # process payees
    df_ynab[['Payee', 'Category']] = df_ynab['Payee'].apply(transform_payee).apply(pd.Series)
    return df_ynab

In [512]:
known_payees = []

In [513]:
# process processed csv files
for f in result_ynab:
    file_path = f
    df_in = pd.read_csv(file_path)
    df_ynab_out = process_ynab(df_in)
    ynab_file_name = os.path.basename(file_path).replace('_processed', '_ynab')
    ynab_dir_name = os.path.dirname(file_path).replace('processed', 'ynab')
    ynab_full_path = os.path.join(ynab_dir_name, ynab_file_name)
    df_ynab_out.to_csv(ynab_full_path, index=False)
    
    for payee in list(df_ynab_out['Payee']):
        if payee not in known_payees:
            known_payees.append(payee)
            
known_payees = list(set(known_payees))
    